# Face Mask Detection - Haar Cascades and NN

In [1]:
import cv2
from keras.models import Model, load_model
import numpy as np

Using TensorFlow backend.


## Step 1: Download Data
### MobileNet Single Shot SSD files
Pre-trained model.

In [2]:
import os
import wget

out_dir = f'{os.getcwd()}\\Data\\facemask_data'

url = 'https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt'
localnetworkfile = out_dir + '\\deploy.prototxt'

if os.path.exists(localnetworkfile):
    print("File exists")
else:
    wget.download(url, out=out_dir)
    print("Download Finished")

File exists


In [3]:
url = 'https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/mobilenet_iter_73000.caffemodel'
localcaffe = out_dir + '\\mobilenet_iter_73000.caffemodel'

if os.path.exists(localcaffe):
    print("File exists")
else:
    wget.download(url, out=out_dir)
    print("Download Finished")

File exists


## Step 2: Load Pre-Trained Models

In [10]:
# Labels of Network.
classNames = { 0: 'background',
    1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat',
    5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair',
    10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse',
    14: 'motorbike', 15: 'person', 16: 'pottedplant',
    17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor' }

# Capture accesses the video feed. The "0" is the number of your video device, in case you have multiple.
cap = cv2.VideoCapture(0)
if cap.isOpened() == True:
    print("Video stream open.")
else:
    print("Problem opening video stream.")

#Load the Caffe model 
net = cv2.dnn.readNetFromCaffe(localnetworkfile, localcaffe)

# load the best face mask model
mask_model = load_model(out_dir+"./model1-006.model")

Video stream open.


## Step 3: Start Camera Inference

In [11]:

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        frame_resized = cv2.resize(frame,(300,300)) # resize frame for prediction

        # MobileNet requires fixed dimensions for input image(s)
        # so we have to ensure that it is resized to 300x300 pixels.
        # set a scale factor to image because network the objects has differents size. 
        # We perform a mean subtraction (127.5, 127.5, 127.5) to normalize the input;
        # after executing this command our "blob" now has the shape:
        # (1, 3, 300, 300)
        blob = cv2.dnn.blobFromImage(frame_resized, 0.007843, (300, 300), (127.5, 127.5, 127.5), False)
        #Set to network the input blob 
        net.setInput(blob)
        
        # Prediction of Mobilenet
        detections = net.forward()

        #Size of frame resize (300x300)
        cols = frame_resized.shape[1] 
        rows = frame_resized.shape[0]

        #For get the class and location of object detected, 
        # There is a fix index for class, location and confidence
        # value in @detections array .
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2] # Confidence of prediction
            if confidence > 0.2: # Filter prediction
                class_id = int(detections[0, 0, i, 1]) # Class label

                # Object location
                xLeftBottom = int(detections[0, 0, i, 3] * cols)
                yLeftBottom = int(detections[0, 0, i, 4] * rows)
                xRightTop   = int(detections[0, 0, i, 5] * cols)
                yRightTop   = int(detections[0, 0, i, 6] * rows)

                # Factor for scale to original size of frame
                heightFactor = frame.shape[0]/300.0
                widthFactor = frame.shape[1]/300.0
                # Scale object detection to frame
                xLeftBottom = int(widthFactor * xLeftBottom)
                yLeftBottom = int(heightFactor * yLeftBottom)
                xRightTop   = int(widthFactor * xRightTop)
                yRightTop   = int(heightFactor * yRightTop)
                
                height = yRightTop - yLeftBottom
                width = xRightTop - xLeftBottom
                print(height)
                print(width)
                y = yLeftBottom
                x = xRightTop
                print(y)
                print(x)

                # Draw location of object  
                #cv2.rectangle(frame, (xLeftBottom, yLeftBottom), (xRightTop, yRightTop), (0, 255, 0))

                # grab the scaled image content of the bounding box for later
                #persons = frame[yLeftBottom:yRightTop, xLeftBottom:xRightTop]

                # Draw label and confidence of prediction in frame resized
                if class_id == 15:

                    # draw rectangle around detected persons
                    cv2.rectangle(frame, (xLeftBottom, yLeftBottom), (xRightTop, yRightTop), (0, 255, 0))
                    mobilenetlabel = classNames[class_id] + ": " + str(confidence)
                    #labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

                    # grab the scaled image content of the bounding box for mask network
                    persons = frame[y:y+height, x:x+width]
                    #persons = frame[y:y+h, x:x+w]
                    
                    # pass the person image into the face mask prediction
                    for person in persons:
                        rerect_sized = cv2.resize(person,(150,150))
                        normalized = rerect_sized/255.0
                        reshaped = np.reshape(normalized,(1,150,150,3))
                        reshaped = np.vstack([reshaped])
                        result = mask_model.predict(reshaped)

                        masklabel = np.argmax(result,axis=1)[0]
                        # assemble label text
                        label = str(mobilnetlabel) + str(results[masklabel])
                        
                        # draw white boxes on top of persons
                        yLeftBottom = max(yLeftBottom, labelSize[1])
                        cv2.rectangle(frame, (xLeftBottom, yLeftBottom - labelSize[1]), xLeftBottom + labelSize[0], (yLeftBottom + baseLine),(255, 255, 255), cv2.FILLED)
                        # draw boxes???
                        cv2.rectangle(frame,(x,y),(x+w,y+h),GR_dict[label],2)
                        cv2.rectangle(frame,(x,y-40),(x+w,y),GR_dict[label],-1)
                        # draw assembled label
                        cv2.putText(frame, label, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
                        print('bla')
                    

                    
        cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
        cv2.imshow("frame", frame)


        if cv2.waitKey(1) >= 0:  # Break with any key 
            break
    cap.release()
    cv2.destroyAllWindows()
    


456
497
478
138


ValueError: cannot reshape array of size 22500 into shape (1,150,150,3)

In [12]:
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
                    # pass the person image into the second neural network for face mask prediction
                    for person in persons:
                        rerect_sized = cv2.resize(person,(150,150))
                        normalized = rerect_sized/255.0
                        reshaped = np.reshape(normalized,(1,150,150,3))
                        reshaped = np.vstack([reshaped])
                        result = mask_model.predict(reshaped)

                        masklabel = np.argmax(result,axis=1)[0]
                        # assemble label text
                        label = str(mobilnetlabel) + str(results[masklabel])
                        
                        # draw white boxes on top of persons
                        yLeftBottom = max(yLeftBottom, labelSize[1])
                        cv2.rectangle(frame, (xLeftBottom, yLeftBottom - labelSize[1]),
                                      xLeftBottom + labelSize[0], yLeftBottom + baseLine),
                                      (255, 255, 255), cv2.FILLED)
                        # draw boxes???
                        cv2.rectangle(frame,(x,y),(x+w,y+h),GR_dict[label],2)
                        cv2.rectangle(frame,(x,y-40),(x+w,y),GR_dict[label],-1)
                        # draw assembled label
                        cv2.putText(frame, label, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
